In [1]:
% load_ext autoreload
% autoreload 2
% matplotlib inline

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import scipy as sp
import seaborn as sns
np.set_printoptions(precision=5, linewidth=120)
from tqdm import *
from drift_qec.A import *

/Users/yan/.miniconda/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [37]:
from scipy.special import binom as bn

def all_rates(C, n, wx, wy, wz):
    px, py, pz = np.diag(C)
    rates = {}
    for x in range(wx+1):
        for y in range(wy+1):
            for z in range(wz+1):
                errstring = "x" * x + "y" * y + "z" * z
                rates[errstring] = bn(n, x+y+z) \
                    * (px ** x) * (py ** y) * (pz ** z) \
                    * ((1 - px - py - pz) ** (n - x - y - z))
    rates["fail"] = 1 - np.sum(rates.values())
    return rates

In [38]:
channel = Channel(kx=0.99998, ky=0.00001, kz=0.00001, Q=FIXEDQ,
                  n=1, stacklength=2001,
                  Vdecayfn = lambda V, t: V / np.sqrt(t))

In [42]:
all_rates(0.05*channel.C, 15, 3, 1, 1)

{'': 0.46331534863660179,
 'fail': 0.14285179084439514,
 'x': 0.01336103455539922,
 'xx': 0.00017980852054096105,
 'xxx': 1.4979747942420265e-06,
 'xxxy': 1.7277900810782496e-07,
 'xxxyz': 4.6594944606536299e-09,
 'xxxz': 5.5087225944719098e-08,
 'xxy': 2.9956949084535648e-05,
 'xxyz': 1.1016508616683923e-06,
 'xxz': 9.5511905115487436e-06,
 'xy': 0.0035958647071473743,
 'xyz': 0.0001910075716567343,
 'xz': 0.0011464715173364714,
 'y': 0.26719797518044675,
 'yz': 0.022927481159052094,
 'z': 0.085190877016442956}